In [1]:
import ROOT
import numpy as np

Welcome to JupyROOT 6.30/02


In [2]:
data_values = np.genfromtxt("homework-dataset.csv")

In [3]:
#--------------------------------
# make model
#-------------------------------
# this is the distribution we'll be looking at
mass = ROOT.RooRealVar("mass", "mass", 0, 200)

# background variables
c = ROOT.RooRealVar("c", "c", -0.1, -1, 0)

# signal variables
# we want to find a particle in the 100 --> 150 GeV range
mean = ROOT.RooRealVar("mean", "mean",    100, 100, 150)
sigma = ROOT.RooRealVar("sigma", "sigma", 1, 0, 40)


# what fraction is signal
# initial guess = half
fsig = ROOT.RooRealVar("fsig", "fsig", 0.5, 0, 1)

# background
sig = ROOT.RooGaussian("sig", "sig",
                       mass, mean, sigma)
# background
bkg = ROOT.RooExponential("bkg", "bkg",
                          mass, c)

# total model is the sum of those
model = ROOT.RooAddPdf("model", "model",
                       ROOT.RooArgList(sig, bkg), fsig)

[#0] WARNING:InputArguments -- The parameter 'sigma' with range [0, 40] of the RooGaussian 'sig' exceeds the safe range of (0, inf). Advise to limit its range.


In [4]:
#------------------------------
# fit model to data
#------------------------------
data = ROOT.RooDataSet.from_numpy({'mass': data_values}, [mass])
model.fitTo(data)

# print values
print(mean)
print(sigma)
print(c)
print(fsig)

RooRealVar::mean = 129.744 +/- 0.310208  L(100 - 150) 

RooRealVar::sigma = 12.3019 +/- 0.292411  L(0 - 40) 

RooRealVar::c = -0.0251849 +/- 0.000117131  L(-1 - 0) 

RooRealVar::fsig = 0.0505672 +/- 0.00127827  L(0 - 1) 

[#0] WARNING:DataHandling -- RooDataSet.from_numpy() Ignored 636 out-of-range events
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (sig,bkg)
Minuit2Minimizer: Minimize with max-calls 2000 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 470651.759200936765
Edm   = 1.74249795643042809e-06
Nfcn  = 400
c	  = -0.0251849	 +/-  0.000116751	(limited)
fsig	  = 0.0505672	 +/-  0.00127302	(limited)
mean	  = 129.744	 +/-  0.310299	(limited)
sigma	  = 12.3019	 +/-  0.291639	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       710480.3221 Edm =       227449.5745 NCalls =     19
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : 498383.5334
  Edm           : 33556.507
  Internal parameters:	[      0.927295218                0     -1.370461484     0.8049223377]	
  Internal gradient  :	[     -115282.3162      15037.77132      3857.955329     -11506.55608]	
  Internal covariance matrix:
[[  8.8329672e-06              0              0              0]
 [              0  2.6749729e-05              0              0]
 [              0              0  0.00010318776              0]
 [              0              0              0  6.9870537e-05]]]
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 498383.5334
  Edm 

In [5]:
#-------------------------------
# plot results
#-------------------------------
hist_data = ROOT.TH1F("data_hist", "", 50, 0, 200)
for v in data_values:
    hist_data.Fill(v)

# plot bkg only, sig only, and combo
bkg_only_graph = ROOT.TGraph()
sig_only_graph = ROOT.TGraph()
model_graph =    ROOT.TGraph()

for v in np.linspace(0, 200, 1000):
    mass.setVal(v)
    n = bkg_only_graph.GetN()
    
    bkg_only_graph.SetPoint(n, v, bkg.getVal(mass))
    sig_only_graph.SetPoint(n, v, sig.getVal(mass))
    model_graph.SetPoint(   n, v, model.getVal(mass))


bkg_only_graph.Scale(hist_data.GetBinWidth(1)*len(data_values)*(1.0-fsig.getVal()))
sig_only_graph.Scale(hist_data.GetBinWidth(1)*len(data_values)*fsig.getVal())
model_graph.Scale(hist_data.GetBinWidth(1)*len(data_values))


leg = ROOT.TLegend(0.6, 0.6, 0.88, 0.88)
leg.AddEntry(hist_data, "Data", "lep")
leg.AddEntry(bkg_only_graph, "Background only", "l")
leg.AddEntry(sig_only_graph, "Signal only", "l")
leg.AddEntry(model_graph, "Combined model", "l")


hist_data.GetXaxis().SetTitle("Mass [GeV]")
hist_data.GetYaxis().SetTitle("Events / 4 GeV")
hist_data.SetStats(0)
hist_data.SetLineWidth(2)

bkg_only_graph.SetLineColor(ROOT.kRed)
sig_only_graph.SetLineColor(ROOT.kMagenta)
model_graph.SetLineColor(ROOT.kBlack)


c = ROOT.TCanvas()
hist_data.Draw("e0")
bkg_only_graph.Draw("c, same")
sig_only_graph.Draw("c, same")
model_graph.Draw("c, same")

leg.Draw("same")
c.SaveAs("homework-solutions.png")

Info in <TCanvas::Print>: png file homework-solutions.png has been created
